In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [4]:
import torch

if torch.cuda.is_available(): #This checks if GPU is available and determines which device to run on. So if GPU unavailable, runs on CPU
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

# torch.cuda.device_count() 
# We can assign various parts of the network across multiple GPUs if possible. Cannot cross interact 

cuda:0


In [5]:
import torch.nn as nn
import torch.nn.functional as F 

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) # 1 input, 32 convolution feature output, 5 kernal size
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        #self.fc1 = nn.Linear(??, 512)
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        '''This function is used to find the size of the output of the convolutional layers. There may be a better way to do this, but the tutorial didn't know'''
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        # print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1,self._to_linear) #flattens output of convolution 
        x = F.relu(self.fc1(x))
        x = self.fc2(x) # Don't run relu on last layer. Why? Idk
        return F.softmax(x, dim=1) # Activation function on the output 

model = Net().to(device) 